In [1]:
import os
os.chdir('/content/drive/MyDrive/MVA/KKML')

# Kernel Methods: Challenge

Julia Linhart, Roman Castagné, Louis Bouvier

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from functools import partial
from scipy.spatial import distance_matrix
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.model_selection import GridSearchCV
import cvxpy as cp
import warnings
import time
from itertools import product
from numba import jit

from utils import run_model, write_csv


warnings.filterwarnings("ignore", category=DeprecationWarning)

# I) Preprocessing

In [3]:
data_folder = 'data' # 'machine-learning-with-kernel-methods-2021'

X_train_1 = pd.read_csv(f'{data_folder}/Xtr2_mat100.csv', sep = ' ', index_col=False, header=None)
y_train_1 = pd.read_csv(f'{data_folder}/Ytr2.csv')

In [4]:
y_train_1.describe()

,Id,Bound
count,2000.000000,2000.000000
mean,4999.500000,0.498500
std,577.494589,0.500123
min,4000.000000,0.000000
25%,4499.750000,0.000000
50%,4999.500000,0.000000
75%,5499.250000,1.000000
max,5999.000000,1.000000


In [5]:
y_train_1 = np.array(y_train_1)[:,1]

In [6]:
X_train_1.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99
count,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,...,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000
mean,0.010565,0.010201,0.010375,0.011587,0.011609,0.010707,0.009359,0.011957,0.009571,0.010582,0.009424,0.009793,0.012848,0.012092,0.011196,0.010364,0.009875,0.010962,0.010185,0.008342,0.010734,0.010038,0.011554,0.008995,0.010283,0.008647,0.008886,0.008826,0.007821,0.009761,0.008533,0.011864,0.009299,0.010641,0.009560,0.008929,0.010217,0.009641,0.009880,0.010038,...,0.009511,0.010614,0.011957,0.009641,0.011772,0.009500,0.008783,0.010005,0.010870,0.009147,0.013565,0.010587,0.009793,0.010908,0.009500,0.009772,0.009103,0.010147,0.008587,0.010538,0.010897,0.008913,0.008630,0.008380,0.009016,0.011478,0.008832,0.009989,0.010587,0.008625,0.007951,0.009457,0.008554,0.009283,0.008261,0.009614,0.011141,0.009777,0.008217,0.008565
std,0.012278,0.010723,0.011467,0.011453,0.012182,0.010478,0.009789,0.012444,0.013805,0.013652,0.012934,0.011163,0.027178,0.018160,0.011200,0.010356,0.010089,0.019951,0.010631,0.009920,0.011238,0.010962,0.011475,0.009723,0.010922,0.009933,0.009622,0.009861,0.010099,0.010628,0.009945,0.010829,0.010358,0.010460,0.011039,0.009612,0.010705,0.012258,0.020208,0.011266,...,0.010436,0.011172,0.012915,0.010912,0.011305,0.016977,0.014644,0.012108,0.011800,0.009647,0.011868,0.011752,0.013102,0.010237,0.009652,0.009687,0.011871,0.010457,0.012348,0.011010,0.011005,0.010695,0.009248,0.010494,0.009279,0.011204,0.010571,0.015973,0.009745,0.011904,0.009605,0.009701,0.009350,0.009741,0.012341,0.010338,0.010863,0.010402,0.009709,0.009283
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000

In [ ]:
X_train_1 = np.array(X_train_1)
X_train_1 = (X_train_1 - X_train_1.mean(axis=0))/X_train_1.std(axis=0)

# II) First linear models of the mat100 input

## A) Logistic regression

In [13]:
from utils import run_model

run_model('logreg')

/content/drive/My Drive/MVA/KKML/functions.py:103: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  inv_hess, _, _, _ = np.linalg.lstsq(hess, np.eye(hess.shape[0]))
/content/drive/My Drive/MVA/KKML/functions.py:111: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  inv_hess, _, _, _ = np.linalg.lstsq(hess, np.eye(hess.shape[0]))


Accuracy on train set 0: 0.62
Accuracy on test set 0 : 0.56
Accuracy on train set 1: 0.60
Accuracy on test set 1 : 0.59
Accuracy on train set 2: 0.70
Accuracy on test set 2 : 0.66


In [ ]:
from utils import write_csv

ids = np.arange(all_y_eval.shape[0])
filename = "results/submission_log_reg.csv"

# write_csv(ids, all_y_eval, filename)

## B) Ridge regression

In [10]:
run_model('rr')

Accuracy on train set 0: 0.65
Accuracy on test set 0 : 0.60
Accuracy on train set 1: 0.64
Accuracy on test set 1 : 0.57
Accuracy on train set 2: 0.73
Accuracy on test set 2 : 0.69


# III) Kernel baselines 

## A) Gaussian Kernel

### a) Kernel Ridge Regression

In [10]:
## run kernel ridge regression with gaussian kernel
run_model('krr',kernel='gaussian',prop_test=0.2)

{'kernel': 'gaussian', 'lamb': 0.1, 'sigma': 0.5789473684210527}
Accuracy on train set 0: 1.00
Accuracy on test set 0 : 0.57
{'kernel': 'gaussian', 'lamb': 0.1, 'sigma': 0.6578947368421053}
Accuracy on train set 1: 1.00
Accuracy on test set 1 : 0.59
{'kernel': 'gaussian', 'lamb': 0.1, 'sigma': 0.5}
Accuracy on train set 2: 1.00
Accuracy on test set 2 : 0.67


### b) Kernel SVM

In [9]:
## run kernel SVM with gaussian kernel
run_model('ksvm',kernel='gaussian',prop_test=0.2)

{'kernel': 'gaussian', 'lamb': 1e-07, 'sigma': 100.0}
Accuracy on train set 0: 0.99
Accuracy on test set 0 : 0.62
{'kernel': 'gaussian', 'lamb': 1e-10, 'sigma': 1.0}
Accuracy on train set 1: 1.00
Accuracy on test set 1 : 0.60
{'kernel': 'gaussian', 'lamb': 1e-07, 'sigma': 100.0}
Accuracy on train set 2: 0.99
Accuracy on test set 2 : 0.72


## B) Spectrum kernel

In [ ]:
from kernels import Spectrum_kernel

In [ ]:
## run kernel SVM with gaussian kernel
run_model('ksvm',kernel='spectrum',sequence=True,prop_test=0.2)

### a) Kernel SVM


## C) Substring kernel

In [13]:
from kernels import substring_similarity, substring_kernel

In [12]:
# Run similarity between two strings
t0 = time.time()
# k = substring_similarity("ATGCATGATGCATG", "ATGCATCATGATGT", 3, 1.)
# k = substring_similarity("ATGC", "ATGC", 3, 0.7)
k = substring_similarity("cat", "cat", 1, 0.7)
k_expected = 2 * 0.7 ** 4 + 0.7 ** 6
print(f"Time to compute : {time.time() - t0:.4f}s")
print(f"Value : {k}")
print(f"Expected value : {k_expected}")

Time to compute : 2.0617s
Value : 0.9799999999999999
Expected value : 0.5978489999999999


In [14]:
# Run kernel computation between N strings
X = pd.read_csv(f'{data_folder}/Xtr0.csv', sep = ',').to_numpy()
X = X[:100,1]
t0 = time.time()
K = substring_kernel(X, X, k=5, lambd=0.7)
print(f"Time to compute K : {time.time() - t0:.2f}s")
# print(K)

Time to compute K : 76.18s


## D) Mismatch kernel

In [ ]:
class Node(object):
    def __init__(self, parent, letter, m=1):
        self.m = m
        self.parent = parent
        self.letter = letter
        self.sequence = self.parent.get_sequence()+self.letter
        self.pointers = []
        self.mismatch_per_pointer = []
        
    def get_sequence():
        return self.sequence
    
    def get_pointers():
        return self.pointers

In [ ]:
class Tree(Node):
    def __init__(self,k,m):
        self.root